## Glacier Flowlines

OGGM's default dynamic model is a flowline model (we'll cover this in session 05). But before we can use this model, we need to define flowlines for our glacier. OGGM supports two types of flowlines: centerlines and elevation band flowlines.

In this notebook, we will focus only on elevation band flowlines, which are currently the default in OGGM. If you are curious about the differences between the two approaches, check out this [OGGM tutorial](https://tutorials.oggm.org/stable/notebooks/tutorials/elevation_bands_vs_centerlines.html) or the [OGGM documentation](https://docs.oggm.org/en/stable/flowlines.html).

In [ ]:
# first we import the packages we need for this session
from oggm import cfg, utils, workflow, tasks 

## OGGM initzial set up

As always, we need to start by setting up the working directory.

In [ ]:
cfg.initialize()

# define working directory
path = 'flowline_working_dir'
utils.mkdir(path, reset=False)  # if you set reset=True, everything will be deleted and you can start from a fresh state
cfg.PATHS['working_dir'] = path

## Start from a preprocessed directory

Thanks to the preprocessed directories, we don’t need to start from scratch every time. We can begin with any glacier using the state we created in the last notebook, which already includes the 2D grid and DEM. For this, we select level 1 of the [preprocessed directories](https://docs.oggm.org/en/stable/shop.html#pre-processed-directories).

In [ ]:
# select the glacier of your choice, you can use the Glims viewer from the first session
rgi_ids = ['RGI60-11.00897']  # Hintereisferner

# we load the outline data from the oggm cluster
prepro_base_url_L1 = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/elev_bands/'
gdirs = workflow.init_glacier_directories(rgi_ids,
                                          from_prepro_level=1,  # here we select level 1
                                          prepro_base_url=prepro_base_url_L1,
                                          prepro_border=80,  # could be 10, 80, 160 or 240
                                         )
gdir = gdirs[0]  # for convenience, we define a single variable for our glacier

<div class="alert alert-warning">
    <b>Task</b>: Take a look in your glacier’s working directory. Do you recognize all the files?
</div>

## Create elevation band flowlines

Before we start coding, please read [this paragraph](https://docs.oggm.org/en/stable/flowlines.html#elevation-bands-flowlines) that explains how elevation band flowlines are created.

<div class="alert alert-warning">
    <b>Task</b>: Can you explain how elevation bands are created in your own words?
</div>

Your answer here:


### Glacier Mask

Now we start creating the actual elevation bands for our selected glacier. The first step is to create a glacier mask on the grid we defined earlier.

In [ ]:
workflow.execute_entity_task(tasks.simple_glacier_masks, 
                             gdirs);

<div class="alert alert-warning">
    <b>Task</b>: Have a look in your glacier's working directory. Is their a new file?
</div>

Now you should now see a new file called `gridded_data.nc`. Let's have a look at it:

In [ ]:
import xarray as xr

ds = xr.open_dataset(gdir.get_filepath('gridded_data'))
ds

We also can plot individual variables with:

In [ ]:
ds.topo.plot()

<div class="alert alert-warning">
    <b>Task</b>: Plot all the variables by adapting the line of code above, and try to explain what each of them is showing.
</div>

Your answer here:


### elevation band flowline

Finally, we create our elevation band flowline:

In [ ]:
workflow.execute_entity_task(tasks.elevation_band_flowline, 
                             gdirs);

<div class="alert alert-warning">
    <b>Task</b>: Have a look in your glacier's working directory. Is their a new file?
</div>

You should see a new file called `elevation_band_flowline.csv`. Let's have a look at it:

In [ ]:
import pandas as pd

df_eb = pd.read_csv(gdir.get_filepath('elevation_band_flowline'))
df_eb

<div class="alert alert-warning">
    <b>Task</b>: What do the individual columns stand for?
</div>

Your answer here: 

### fixed dx elevation band flowline

As you can see above, the created flowline has non-uniform `dx` values. However, our dynamic flowline model requires constant `dx` values to work. Let’s fix that:

In [ ]:
workflow.execute_entity_task(tasks.fixed_dx_elevation_band_flowline, 
                             gdirs);

<div class="alert alert-warning">
    <b>Task</b>: Have a look in your glacier's working directory. Is their a new file?
</div>

You will notice that a new file called `inversion_flowlines.pkl` has appeared in your working directory. We will explain why it is named this way in session 04. For now, let’s just take a look at it:

In [ ]:
fl_inv = gdir.read_pickle('inversion_flowlines')[0]  # with zero we select the first and only flowline, with centerlines also multiple flowlines are possible
fl_inv

You can see that this is an `Centerline` object. Whenever you encounter a new object in Python, you can check its available methods and properties using the dir command:

In [ ]:
dir(fl_inv)

We can look at the spacing of the flowline with:

In [ ]:
fl_inv.map_dx

This is the same spacing in meters as the grid defined in the previous notebook.

To get a better understanding of our flowline, let’s try to visualize it:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# Plot setup
fig, axs = plt.subplots(2, 1, figsize=(5, 5))

# defint two axis
ax_xz = axs[0]
ax_xy = axs[1]

# define grid points in x direction
distance_along_line_meter = np.arange(fl_inv.nx) * fl_inv.dx * fl_inv.map_dx

# plot the surface height per grid point
ax_xz.plot(distance_along_line_meter, fl_inv.surface_h, '.-')

# plot the numeric gird per grid point
for x, width in zip(distance_along_line_meter, fl_inv.widths_m):
    lower_left_y = (-1) * width / 2
    rect = patches.Rectangle((x, lower_left_y), fl_inv.dx_meter, width,
                             edgecolor='black', facecolor='skyblue')
    ax_xy.add_patch(rect)

# define the axis limits
ax_xy.set_xlim(0, distance_along_line_meter[-1] + fl_inv.dx_meter)
ax_xz.set_xlim(0, distance_along_line_meter[-1] + fl_inv.dx_meter)
ax_xy.set_ylim(-max(fl_inv.widths_m/2),
               max(fl_inv.widths_m/2))

# add labels to the axis
ax_xz.set_ylabel('Height in m')
ax_xy.set_ylabel('Width in m')
ax_xy.set_xlabel('Distance along flowline in m')

plt.show()


In [ ]:
from oggm import graphics
graphics.plot_domain(gdir, figsize=(6, 5))

<div class="alert alert-warning">
    <b>Task</b>: Compare the flowline representation with the actual map view. Does it make sense?
</div>

Your answer here:

### Downstream line

So far, we have only focused on the glacier as defined by its outline. But what happens if the glacier starts to grow? For that, we also need to define a downstream line:

In [ ]:
workflow.execute_entity_task(tasks.compute_downstream_line,
                             gdirs);
workflow.execute_entity_task(tasks.compute_downstream_bedshape,
                             gdirs);

<div class="alert alert-warning">
    <b>Task</b>: Have a look in your glacier's working directory. Is their a new file?
</div>

You’ll see that a new file called `downstream_line.pkl` has appeared in your working directory. Let’s take a look at it:

In [ ]:
dwn = gdir.read_pickle('downstream_line')
dwn

When you compare the `surface_h` values of the downstream line with those from the elevation band flowline above, you will see that they actually start where the flowline ends. Let's plot them:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2.5))

# plot the elevation band flowline heights
distance_along_flowline_meter = np.arange(fl_inv.nx) * fl_inv.dx * fl_inv.map_dx
ax.plot(distance_along_flowline_meter, fl_inv.surface_h, '.-', label='elevation band flowline')

# we need to create x values for the downstream line, we start from the last elevation band flowline grid point
last_eb_grid_point = distance_along_flowline_meter[-1]
downstream_x = np.arange(1, len(dwn['surface_h']) + 1) * fl_inv.map_dx * fl_inv.dx + last_eb_grid_point

# plot the downstream heights
ax.plot(downstream_x, dwn['surface_h'], '.-', label='downstream line')

# add a legend
ax.legend()

# add axis labels
ax.set_ylabel('Height in m')
ax.set_xlabel('Distance along flowline in m');

Additionally, the parameters `bedshapes` and `w0` define the shape of the downstream line. We’ll learn more about the different bed shapes in session 04.

## Recap

- By default, OGGM uses elevation band flowlines.
- These flowlines are created using the glacier outline and a Digital Elevation Model (DEM).
- To allow glaciers to advance, we also defined a downstream line.